In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

d:\CampusX\Miscellaneous topics\MLOps-Bootcamp\4. MLflow_basics\basics\.venv\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
## load the dataset
data=pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
## Split the data into training,validation and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [4]:
train_x = train.drop(['quality'], axis=1).values
train_y = train['quality'].values.ravel()

## Validation dataset
test_x = test.drop(['quality'], axis=1).values
test_y = test[['quality']].values.ravel()

## Splitting this into train and validation 
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.20, random_state=42)
signature = infer_signature(train_x, train_y)

In [5]:
np.mean(train_x, axis=0)

array([6.86621852e+00, 2.80377808e-01, 3.32597005e-01, 6.42164738e+00,
       4.55513955e-02, 3.53556841e+01, 1.38792376e+02, 9.94074221e-01,
       3.18919333e+00, 4.88396869e-01, 1.05005673e+01])

In [ ]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate = params["lr"], momentum = params["momentum"]
    ),
    loss = "mean_squared_error",
    metrics = [keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y),
                  epochs = epochs,
                  batch_size = 64)
        
        ## Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)

        eval_rmse = eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "model": model}

In [7]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs = 3,
        train_x = train_x,
        train_y = train_y,
        valid_x = valid_x,
        valid_y = valid_y,
        test_x = test_x,
        test_y = test_y,
    )
    return result

In [8]:
space={
    "lr":hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum":hp.uniform("momentum", 0.0, 1.0)
}

In [11]:
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2025/11/06 17:22:23 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 44s 988ms/step - loss: 35.4418 - root_mean_squared_error: 5.9533
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.5297 - root_mean_squared_error: 2.9420    
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - loss: 3.0763 - root_mean_squared_error: 1.7539 - val_loss: 0.7858 - val_root_mean_squared_error: 0.8865

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.7478 - root_mean_squared_error: 0.8647
27/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7084 - root_mean_squared_error: 0.8416 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6551 - root_mean_squared_error: 0.8094 - val_loss: 0.5798 - val_root_mean_squared_error: 0.7615

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.4181 - root_mean_squared_error: 0.6466
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5547 - root_mean_squared_error: 0.7445 
46

2025/11/06 17:22:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.

2025/11/06 17:24:50 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\subhadip\AppData\Local\Temp\tmp6j6nd770\model, flavor: tensorflow). Fall back to return ['tensorflow==2.20.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 



🏃 View run bemused-sow-328 at: http://localhost:5001/#/experiments/433580776526248839/runs/721aaa91fd544efcbf3822b64b91b1e8

🧪 View experiment at: http://localhost:5001/#/experiments/433580776526248839

Epoch 1/3                                                                       

 1/46 ━━━━━━━━━━━━━━━━━━━━ 43s 970ms/step - loss: 41.3948 - root_mean_squared_error: 6.4339
 3/46 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 41.3180 - root_mean_squared_error: 6.4279  
30/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 36.5340 - root_mean_squared_error: 6.0375 
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 25.4467 - root_mean_squared_error: 5.0445 - val_loss: 11.9614 - val_root_mean_squared_error: 3.4585

Epoch 2/3                                                                       

 1/46 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - loss: 13.4583 - root_mean_squared_error: 3.6685
24/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.3473 - root_mean_squared_error: 3.2103 
44/46 ━━━━━━━━━━━━━━━━━━━━ 0s 

2025/11/06 17:25:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.

2025/11/06 17:27:07 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\subhadip\AppData\Local\Temp\tmp2y08tmz6\model, flavor: tensorflow). Fall back to return ['tensorflow==2.20.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 



...

🏃 View run secretive-donkey-135 at: http://localhost:5001/#/experiments/433580776526248839/runs/c9f033a9a0c049ae98e14c57332be884

🧪 View experiment at: http://localhost:5001/#/experiments/433580776526248839   

Epoch 1/3                                                                       

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2:28 3s/step - loss: 31.3481 - root_mean_squared_error: 5.5989
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 21.2003 - root_mean_squared_error: 4.5528
14/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 16.1336 - root_mean_squared_error: 3.9361 
24/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 12.4809 - root_mean_squared_error: 3.4252
36/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 10.0854 - root_mean_squared_error: 3.0497
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 8.7964 - root_mean_squared_error: 2.8298 
46/46 ━━━━━━━━━━━━━━━━━━━━ 7s 91ms/step - loss: 3.8291 - root_mean_squared_error: 1.9568 - val_loss: 0.8771 - val_root_mean_squared_error: 0.9366

Epoch 2/3              

2025/11/06 17:29:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.

2025/11/06 17:32:00 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\subhadip\AppData\Local\Temp\tmpp90v9u6h\model, flavor: tensorflow). Fall back to return ['tensorflow==2.20.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 



🏃 View run clumsy-jay-147 at: http://localhost:5001/#/experiments/433580776526248839/runs/95b14c7cc0934640bbc80e1a9d593a3d

🧪 View experiment at: http://localhost:5001/#/experiments/433580776526248839   

100%|██████████| 4/4 [09:26<00:00, 141.58s/trial, best loss: 0.7548835873603821]


2025/11/06 17:32:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Best parameters: {'lr': np.float64(0.022895212772029373), 'momentum': np.float64(0.6987937825625921)}
Best eval rmse: 0.7548835873603821
🏃 View run gifted-sloth-278 at: http://localhost:5001/#/experiments/433580776526248839/runs/44ef59e2d8164878a1eda1278559f49a
🧪 View experiment at: http://localhost:5001/#/experiments/433580776526248839


In [ ]:
 # Load model as a PyFuncModel.
model_uri = 'mlruns:/433580776526248839/44ef59e2d8164878a1eda1278559f49a'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

In [ ]:
## Register in the model registry
mlflow.register_model(model_uri, "wine-quality")